In [ ]:
import pandas as pd
import numpy as np

sales_real =  pd.DataFrame.from_csv('sales_march_v2.csv').reset_index()
sales_avg =  pd.DataFrame.from_csv('sales_march_moving_average_v2.csv').reset_index()
sales_ml =  pd.DataFrame.from_csv('sales_march_ml_v2.csv').reset_index()
#real_sales['Salesdate'] = pd.to_datetime(real_sales['Salesdate'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
sales_with_ml = pd.merge(sales_real, sales_ml, on=['Locationid','PLU','Year','Month','Day'], how='inner')
sales_with_ml_avg = pd.merge(sales_with_ml, sales_avg, on=['Locationid','PLU','Year','Month','Day'], how='inner')
sales_with_ml_avg = sales_with_ml_avg.rename(columns={'Quantity_x': 'real', 'Quantity_y':'ml', 'Quantity':'avg'})

In [ ]:
#products = sales_with_ml_avg['PLU'].unique().tolist()
#locations = sales_with_ml_avg['Locationid'].unique().tolist()

In [ ]:
correl = sales_with_ml_avg[['real','ml','avg']].corr()
all_population = correl*correl
all_population

In [ ]:
corr_by_groups = sales_with_ml_avg.groupby(['Locationid','PLU'])['real', 'ml', 'avg'].corr()
#corr_by_groups = corr_by_groups*corr_by_groups

In [ ]:
d = corr_by_groups.reset_index()
d = d[d['level_2']=='real']
c = d[(d['Locationid']==40)&(d['PLU']==2459)]

In [ ]:
c*c

In [ ]:

rows_all = len(d.index)
print("Number of all observations = %d" % (rows_all))

ml_better = d[d['ml'] > d['avg']]
ml_diff = ml_better['ml'] - ml_better['avg']
rows_ml = len(ml_better.index)
print("ML is better in %d observations of %d (%d%%)" % (rows_ml, rows_all, rows_ml/rows_all*100))
print("ML performance is better in %d%% for above observations" % (ml_diff.mean()*100))

avg_better = d[d['ml'] < d['avg']]
avg_diff = avg_better['avg'] - avg_better['ml']
rows_avg = len(avg_better.index)
print("AVG is better in %d observations of %d (%d%%)" % (rows_avg, rows_all, rows_avg/rows_all*100))
print("AVG performance is better in %d%% for above observations" % (avg_diff.mean()*100))

In [ ]:
c = d
c['diff'] = c['ml'] - c['avg']
c['diff'] = c['diff'].abs()

In [ ]:
# best ML
c[(c['ml']>0)&(c['avg']<c['ml'])].sort_values(['diff', 'ml'], ascending=(False,False)).head(5)

In [ ]:
# best AVG
c[(c['avg']>0)&(c['avg']>c['ml'])].sort_values(['diff', 'avg'], ascending=(False,False)).head(5)


In [ ]:
# best ML
best_ml_locations = [484,84,174,268,40]
best_ml_recipes = [1000001486,2413,2916,3383,2459]
s = sales_with_ml_avg[(sales_with_ml_avg['Locationid'].isin(best_ml_locations)) & (sales_with_ml_avg['PLU'].isin(best_ml_recipes))]
s = s.drop(['index','index_y','index_x'], axis=1)
s = s[['Locationid','PLU','Year','Month','Day','real','ml','avg']]
s.to_csv('best_ml_prediction.csv', sep=',')

In [ ]:
# best AVG
best_avg_locations = [227,261,75,195,272]
best_avg_recipes = [2927,4122,354213,353808,4144]
s = sales_with_ml_avg[(sales_with_ml_avg['Locationid'].isin(best_avg_locations)) & (sales_with_ml_avg['PLU'].isin(best_avg_recipes))]
s = s.drop(['index','index_y','index_x'], axis=1)
s = s[['Locationid','PLU','Year','Month','Day','real','ml','avg']]
s.to_csv('best_avg_prediction.csv', sep=',')

cs = s.groupby(['Locationid','PLU'])['real', 'ml', 'avg'].corr()
cs

cs = cs*cs
cs